In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
load_dotenv()

from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.runnables import  RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
rag_prompt_with_history = ChatPromptTemplate.from_messages(
    [
        ("system", """주어진 Context만 근거로 간결하고 정확하게 답하도록 해라.
         
         [Context]
         {context}"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


In [28]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
collection_name = "samsung2025"
vectorstore = Chroma(
    embedding_function= embeddings,
    persist_directory="../7_vectorstore/samsung_2025_db",
    collection_name=collection_name
)


In [29]:
ret_similarity = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":10}
)
ret_similarity.invoke("삼성전자")

[Document(id='7ef39294-b0e0-4bf5-8386-abb0aa455d04', metadata={'total_pages': 87, 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'trapped': '/False', 'page_label': '5', 'moddate': '2025-09-04T16:51:11+09:00', 'creationdate': '2025-07-10T16:11:16+09:00', 'page': 4, 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)'}, page_content='삼성전자 지속가능경영보고서 2025\n05\nOur Company AppendixFacts & Figures PrinciplePlanet People\n회사소개\nAbout Us\n삼성전자주식회사(이하 삼성전자)는 인재와 기술을 기반으로 최고의 제품과 서비스를 창출함으로써 인류사회에 기여하는 글로벌 초일류 기업을 지향합니다. 이를 위해 삼성전자가 지켜나갈 약속인 5가지 경영원칙 을 세부원칙과 \n행동지침으로 구체화하고, 삼성전자 임직원이 지켜야 할 행동규범 으로 제정하여 모든 경영활동의 기준으로 삼고 있습니다. 앞으로도 삼성전자는 조직문화에 5가지 핵심가치 를 내재화하여 지속적으로 성장해갈 것입니다.\n사업부문 및 글로벌 네트워크 소개\n삼성전자는 제품 특성에 따라 DX(Device eXperience)와 DS(Device Solutions) 2개 부문으로 나뉘어 독립적으로 운영되고 있습니다. DX부문은 스마트폰, 네트워크 시스템, 컴퓨터, TV, 냉장고, 세탁기, 에어컨, 의료기기 등 완제품을 \n생산·판매하고 있으며, DS부문은 메모리 사업, Foundry 사업, System LSI 사업으로 구성되어 DRAM, NAND Flash, 모바일AP 등의 반도체 부품을 생산

In [30]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

model_name = "cross-encoder/ms-marco-MiniLM-L6-v2"
max_length = 512
hf_ce = HuggingFaceCrossEncoder(
    model_name=model_name, model_kwargs={"device": "cuda", "max_length": max_length}
)

reranker = CrossEncoderReranker(
    model=hf_ce, top_n=10
)

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
com_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=ret_similarity
)

In [32]:
from langchain_community.document_transformers import LongContextReorder
reorder = LongContextReorder()

In [33]:
def format_docs(docs):
    result = []
    for item in docs:
        result.append(item.page_content)
    
    return "\n\n--\n\n".join(result)

In [34]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
parser = StrOutputParser()

In [35]:

chain = rag_prompt_with_history | model | parser 
chain

ChatPromptTemplate(input_variables=['chat_history', 'context', 'question'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typi

In [36]:
retriever_chain = (
    com_retriever
    | RunnableLambda(lambda docs: reorder.transform_documents(docs))
    | RunnableLambda(format_docs)
)


In [37]:
from operator import itemgetter 
rag_chain =  (
   {
       "context": itemgetter("question") | retriever_chain,
       "question" : itemgetter("question"),
       "chat_history":itemgetter("chat_history")
   }
   | chain 
) 

rag_chain.invoke({
    "question":"삼성의 미래 계획은 어떻게 되나요?",
    "chat_history":[]
})

'삼성전자는 ‘함께가요 미래로! Enabling People’ 비전 아래, 교육 기회 확대와 청년 인재 육성에 집중하며, 2025년에는 삼성 청년SW·AI아카데미 교육 대상을 마이스터고 졸업생까지 확대할 계획입니다. 또한, 국내외 정부기관 및 단체와 협력해 미래세대 교육을 위한 다양한 사회공헌 프로그램을 운영할 예정이며, 지속가능경영과 기술혁신을 통해 탄소중립 달성 등 환경 목표도 추진하고 있습니다.'

In [38]:
from langchain.memory import ChatMessageHistory
chat_history = ChatMessageHistory()

In [39]:
question1 = "삼성의 AI 전략은 뭐야?"
print(f"🧐 사용자: {question1}")

response1 = rag_chain.invoke({
    "question": question1,
    "chat_history": chat_history.messages
})
chat_history.add_user_message(question1)
chat_history.add_ai_message(response1)
print(f"🤖 AI: {response1}\n")


🧐 사용자: 삼성의 AI 전략은 뭐야?
🤖 AI: 삼성의 AI 전략은 Bespoke AI 제품군(예: 패밀리허브 냉장고, 콤보 등)과 모바일 제품군에 삼성 녹스 볼트 보안 솔루션을 적용해 사용자가 쉽고 안전하게 AI HOME을 경험하도록 하는 것입니다. 또한, 2024년 '삼성 청년SW·AI아카데미'에 375억 원을 지원해 2,200명의 청년 소프트웨어 인재를 양성하고, 2025년에는 교육 기회를 마이스터고 졸업생까지 확대하는 등 AI 인재 육성에도 집중하고 있습니다.



In [40]:
question2 = "반도체 쪽은 어때?"
print(f"🧐 사용자: {question2}")
response2 = rag_chain.invoke({
    "question": question2,
    "chat_history": chat_history.messages # 이전 대화가 담긴 기록 전달
})
chat_history.add_user_message(question2)
chat_history.add_ai_message(response2)
print(f"🤖 AI: {response2}")

🧐 사용자: 반도체 쪽은 어때?
🤖 AI: 반도체 분야에서 삼성은 반도체 제품의 전과정 평가(LCA)를 국제표준에 따라 정립하고, PCF(Product Carbon Footprint) 자동화 시스템을 운영해 제품 단위 탄소배출량을 투명하게 관리합니다. 또한, 반도체 기후 컨소시엄(SCC) 활동을 통해 업계 전반의 Scope 3 배출량 산정방법론 표준화와 감축을 주도하며, 협력회사의 온실가스 관리 역량 강화와 감축 목표 수립을 지원하는 등 기후변화 대응과 탄소 감축에 적극적으로 나서고 있습니다.


In [41]:
question3 = "내가 물어본 첫번째 질문이 뭐였나요?"
print(f"🧐 사용자: {question3}")
response3 = rag_chain.invoke({
    "question": question3,
    "chat_history": chat_history.messages # 이전 대화가 담긴 기록 전달
})
chat_history.add_user_message(question3)
chat_history.add_ai_message(response3)
print(f"🤖 AI: {response3}")

🧐 사용자: 내가 물어본 첫번째 질문이 뭐였나요?
🤖 AI: 삼성의 AI 전략은 뭐야?


In [42]:
print(chat_history)

Human: 삼성의 AI 전략은 뭐야?
AI: 삼성의 AI 전략은 Bespoke AI 제품군(예: 패밀리허브 냉장고, 콤보 등)과 모바일 제품군에 삼성 녹스 볼트 보안 솔루션을 적용해 사용자가 쉽고 안전하게 AI HOME을 경험하도록 하는 것입니다. 또한, 2024년 '삼성 청년SW·AI아카데미'에 375억 원을 지원해 2,200명의 청년 소프트웨어 인재를 양성하고, 2025년에는 교육 기회를 마이스터고 졸업생까지 확대하는 등 AI 인재 육성에도 집중하고 있습니다.
Human: 반도체 쪽은 어때?
AI: 반도체 분야에서 삼성은 반도체 제품의 전과정 평가(LCA)를 국제표준에 따라 정립하고, PCF(Product Carbon Footprint) 자동화 시스템을 운영해 제품 단위 탄소배출량을 투명하게 관리합니다. 또한, 반도체 기후 컨소시엄(SCC) 활동을 통해 업계 전반의 Scope 3 배출량 산정방법론 표준화와 감축을 주도하며, 협력회사의 온실가스 관리 역량 강화와 감축 목표 수립을 지원하는 등 기후변화 대응과 탄소 감축에 적극적으로 나서고 있습니다.
Human: 내가 물어본 첫번째 질문이 뭐였나요?
AI: 삼성의 AI 전략은 뭐야?
